In [1]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import division
import math
from cvxpy import *
from numpy.linalg import matrix_rank
import scipy

In [2]:
m = 100
n = 100
smatrix = scipy.sparse.random(m,n,density=0.1, format='coo', dtype=None, random_state=None)
lowrank = np.random.randn(m,n)
rows = np.random.choice(lowrank.shape[0],int(np.ceil(lowrank.shape[0]*0.8)))
lowrank[rows] = np.zeros(n)
M = smatrix + lowrank
S = np.random.randn(m,n)
L = np.random.randn(m,n)
u = np.random.randn(m,n)
rho = 1
a = 1
l = 1

In [3]:
def proxS(x,l):
    prox = np.multiply(np.sign(x),np.maximum(np.subtract(np.absolute(x),l),0))
    return prox

In [4]:
def proxL(x,a):
    U, s, V = np.linalg.svd(x, full_matrices=False)
    #s = np.multiply(np.sign(s),np.maximum(np.subtract(np.absolute(s),a),0))
    #S = np.zeros((m, n), dtype=complex)
    S = np.diag(s)
    S = proxS(S,a)
    prox = np.dot(U, np.dot(S, V))
    return prox

In [5]:
i = 0
iters = 100
for i in range(iters):
    #update S
    xS = S + a*(u + rho*(M-(L+S)))
    S = proxS(xS,l)
    #update L
    xL = L + a*(u + rho*(M-(L+S)))
    L = proxL(xL,a)
    #update u
    u  = u +( M-(L+S))
    i+=1
print i
Mpred = L + S

100


In [6]:
#verify with CVX
# Construct the problem.

# Create two scalar optimization variables.
Lc = Variable(m,n)
Sc = Variable(m,n)

# Create two constraints.
constraints = [Lc+Sc == M]

# Form objective.
obj = Minimize(norm(Lc,'nuc')+l*norm(Sc,1))

# Form and solve problem.
prob = Problem(obj, constraints)
prob.solve()
Mfinal =  Lc.value + Sc.value

print np.linalg.norm(M-Mpred) 
print np.linalg.norm(M-Mfinal) 
print np.linalg.norm(Mpred-Mfinal)
print np.linalg.norm(L-Lc.value)
print np.linalg.norm(S-Sc.value)

0.00405234266595
0.0239153783904
0.0242184890855
0.0360647380089
0.0119592817824
